In [31]:
# imports all libraries
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
# load environment variables
load_dotenv()

True

# Task 1
## Download data and load to pandas dataframe

1. csv format
   * https://www.sistemulenergetic.ro/
   * https://storage.googleapis.com/covid19-open-data/v3/epidemiology.csv

2. json format

   World Air Quality – OpenAQ for Moldova (json format) from https://public.opendatasoft.com/
   
4. API

   Weather forecast for Chisinau from https://openweathermap.org

In [4]:
# 1. read csv format
energ_df = pd.read_csv('./datasets/sistemulenergetic_ro.csv')
print(energ_df.info())
print(energ_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1404 entries, 0 to 1403
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          1404 non-null   object
 1   carbune       1404 non-null   int64 
 2   consum        1404 non-null   int64 
 3   hidro         1404 non-null   int64 
 4   hidrocarburi  1404 non-null   int64 
 5   nuclear       1404 non-null   int64 
 6   eolian        1404 non-null   int64 
 7   productie     1404 non-null   int64 
 8   fotovolt      1404 non-null   int64 
 9   biomasa       1404 non-null   int64 
 10  stocare       1404 non-null   int64 
 11  sold          1404 non-null   int64 
dtypes: int64(11), object(1)
memory usage: 131.8+ KB
None
                  date  carbune  consum  hidro  hidrocarburi  nuclear  eolian  \
0  2025-10-01 08:08:38     1017    6740   1444          1263     1359     146   
1  2025-09-30 20:38:08     1026    7344   1916          1338     1360     254   
2  2

In [5]:
epid_df = pd.read_csv('https://storage.googleapis.com/covid19-open-data/v3/epidemiology.csv')
print(epid_df.info())
print(epid_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12525825 entries, 0 to 12525824
Data columns (total 10 columns):
 #   Column                Dtype  
---  ------                -----  
 0   date                  object 
 1   location_key          object 
 2   new_confirmed         float64
 3   new_deceased          float64
 4   new_recovered         float64
 5   new_tested            float64
 6   cumulative_confirmed  float64
 7   cumulative_deceased   float64
 8   cumulative_recovered  float64
 9   cumulative_tested     float64
dtypes: float64(8), object(2)
memory usage: 955.6+ MB
None
         date location_key  new_confirmed  new_deceased  new_recovered  \
0  2020-01-01           AD            0.0           0.0            NaN   
1  2020-01-02           AD            0.0           0.0            NaN   
2  2020-01-03           AD            0.0           0.0            NaN   
3  2020-01-04           AD            0.0           0.0            NaN   
4  2020-01-05           AD          

In [7]:
# 2. JSON format
url = 'https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/openaq/exports/json/?lang=en&timezone=Europe%2FChisinau'
oaq_df = pd.read_json(url)
print(oaq_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61177 entries, 0 to 61176
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   country                   61177 non-null  object 
 1   city                      32639 non-null  object 
 2   location                  61177 non-null  object 
 3   coordinates               61107 non-null  object 
 4   measurements_parameter    61177 non-null  object 
 5   measurements_sourcename   61177 non-null  object 
 6   measurements_unit         61177 non-null  object 
 7   measurements_value        61177 non-null  float64
 8   measurements_lastupdated  61177 non-null  object 
 9   country_name_en           61050 non-null  object 
dtypes: float64(1), object(9)
memory usage: 4.7+ MB
None


In [10]:
md_oaq = oaq_df[oaq_df['country'] == 'MD']
md_oaq.head()

,country,city,location,coordinates,measurements_parameter,measurements_sourcename,measurements_unit,measurements_value,measurements_lastupdated,country_name_en
54829,MD,N/A,Chisinau,"{'lon': 28.81938, 'lat': 47.021561}",PM2.5,AirNow,µg/m³,13.0,2024-08-09T15:00:00+03:00,"Moldova, Republic of"


In [30]:
# 3. API
API_KEY = os.getenv("OPENWEATHER_API_KEY")
BASE_URL = "https://api.openweathermap.org"

def get_city_geo(cityName: str) -> (int, int):
    api_version = "1.0"
    params = { "q": cityName, "appid": API_KEY }
    resp = requests.get(f"{BASE_URL}/geo/{api_version}/direct", params=params)
    data = resp.json()
    match data:
        case [{ 'lat': lat, 'lon': lon }]:
            return (lat, lon)
        case _:
            raise ValueError("City data not found")

def get_forecast(lat: int, lon: int) -> any:
    api_version = "2.5"
    params = { "appid": API_KEY, "lat": lat, "lon": lon }
    resp = requests.get(f"{BASE_URL}/data/{api_version}/forecast", params)
    return resp.json()

    
lat, lon = get_city_geo("Chisinau")
forecast_data = get_forecast(lat, lon)
forecast_df = pd.DataFrame(forecast_data['list'])
print(forecast_df.info())
print(forecast_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   dt          40 non-null     int64  
 1   main        40 non-null     object 
 2   weather     40 non-null     object 
 3   clouds      40 non-null     object 
 4   wind        40 non-null     object 
 5   visibility  40 non-null     int64  
 6   pop         40 non-null     float64
 7   rain        10 non-null     object 
 8   sys         40 non-null     object 
 9   dt_txt      40 non-null     object 
dtypes: float64(1), int64(2), object(7)
memory usage: 3.3+ KB
None
           dt                                               main  \
0  1759320000  {'temp': 281.75, 'feels_like': 279.03, 'temp_m...   
1  1759330800  {'temp': 280.67, 'feels_like': 277.78, 'temp_m...   
2  1759341600  {'temp': 280.19, 'feels_like': 277.43, 'temp_m...   
3  1759352400  {'temp': 279.64, 'feels_like': 276.51, 'temp_m...  

# Task 2

Scrape date from Bucharest airport for departure flights and download them
to pandas dataframe. Add attribute date and time of scraping

URL: https://www.bucharestairports.ro/

In [49]:

def get_airport_data():
    AIRPORT_URL = "https://www.bucharestairports.ro/"
    # disable ssl warnings
    import urllib3
    urllib3.disable_warnings()
    
    resp = requests.get(AIRPORT_URL, verify=False, timeout=20)
    bs = BeautifulSoup(resp.content)
    
    def get_data_from_table(tip: str, table: list[any]) -> list[dict]:
        data = []
        for row in table:
            colums = row.select('td')
            origine = destinatie = 'Bucuresti (OTP)'
            if tip == 'sosiri':
                origine = colums[1].text
            else:
                destinatie = colums[1].text
            data.append({
                "tip": tip,
                "cursa": colums[0].text,
                "origine": origine,
                "destinatie": destinatie,
                "ora": colums[2].text,
                "ora_est": colums[3].text,
                "status": colums[4].text
            })
        return data
        
    sosiri_table = bs.select('div#sosiri_inner tr')
    plecari_table = bs.select('div#plecari_inner tr')
    return get_data_from_table('sosiri', sosiri_table) + get_data_from_table('plecari', plecari_table)
                        
airport_df = pd.DataFrame(get_airport_data())
airport_df['created_at'] = pd.Timestamp.now(tz="UTC")
print(airport_df.info())
print(airport_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   tip         83 non-null     object             
 1   cursa       83 non-null     object             
 2   origine     83 non-null     object             
 3   destinatie  83 non-null     object             
 4   ora         83 non-null     object             
 5   ora_est     83 non-null     object             
 6   status      83 non-null     object             
 7   created_at  83 non-null     datetime64[us, UTC]
dtypes: datetime64[us, UTC](1), object(7)
memory usage: 5.3+ KB
None
      tip   cursa             origine       destinatie    ora ora_est  \
0  sosiri  FR7669  THESSALONIKI (SKG)  Bucuresti (OTP)  00:20   00:35   
1  sosiri  W43136       BERGAMO (BGY)  Bucuresti (OTP)  00:50   02:16   
2  sosiri  W43146          ROME (FCO)  Bucuresti (OTP)  01:35   02:50   
3  sosi